In [4]:
from dotenv import load_dotenv
import os
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough


In [5]:

load_dotenv()
SERVICE_KEY = os.getenv('OPEN_API_KEY')

# Cache 디렉토리 지정
cache_dir = LocalFileStore("./.cache/")


llm = ChatOpenAI( openai_api_key= SERVICE_KEY,model_name="gpt-3.5-turbo-1106", temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

# splitter = RecursiveCharacterTextSplitter(
#     chunk_size=200,
#     chunk_overlap=50
# )
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100
)

loader = UnstructuredFileLoader("./files/word_test.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings(openai_api_key= SERVICE_KEY)

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)


In [6]:

results = vectorstore.similarity_search("where does winston live")

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

chain.invoke("Describe Victory Mansions")


Victory Mansions is a building with glass doors and a gritty, dusty hallway that smells of boiled cabbage and old rag mats. It has a colored poster of a large face with a caption that reads "BIG BROTHER IS WATCHING YOU." The building has a faulty lift and is seven flights up. Inside the flat, there is a telescreen that cannot be completely shut off.

AIMessageChunk(content='Victory Mansions is a building with glass doors and a gritty, dusty hallway that smells of boiled cabbage and old rag mats. It has a colored poster of a large face with a caption that reads "BIG BROTHER IS WATCHING YOU." The building has a faulty lift and is seven flights up. Inside the flat, there is a telescreen that cannot be completely shut off.')